In [1]:
# Import dependencies
import requests
from config import spotify_token
import pandas as pd
import numpy as np
from collections import defaultdict
import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password
import time

In [2]:
base_url = 'https://api.spotify.com'
headers_dict = {'Content-Type': 'application/json', 'Authorization': f'Bearer {spotify_token}',
           'country': 'US'}


In [11]:
# Create a list of all the 50 category ids
t0 = time.time()
query_url = base_url + f'/v1/browse/categories?limit=50'
results = requests.get(query_url, headers=headers_dict).json()

categories_list = []
for id, category in enumerate(results['categories']['items']):
    categories_list.append(category['id'])

t1 = time.time()
print(f'Run time: {t1-t0} seconds')
categories_list

Run time: 0.1776728630065918 seconds


['toplists',
 '2020',
 'holidays',
 'hiphop',
 'pop',
 'country',
 'workout',
 'rock',
 'latin',
 'mood',
 'rnb',
 'jre_podcast',
 'gaming',
 'shows_with_music',
 'focus',
 'edm_dance',
 'blackhistorymonth',
 'chill',
 'at_home',
 'indie_alt',
 'inspirational',
 'decades',
 'instrumental',
 'alternative',
 'wellness',
 'in_the_car',
 'pride',
 'party',
 'sleep',
 'classical',
 'jazz',
 'roots',
 'soul',
 'sessions',
 'dinner',
 'romance',
 'kpop',
 'punk',
 'regional_mexican',
 'popculture',
 'blues',
 'arab',
 'desi',
 'radar',
 'anime',
 'thirdparty',
 'afro',
 'comedy',
 'metal',
 'caribbean']

In [4]:
# Create a dictionary of categories and playlists
t0 = time.time()
category_playlist_ids = {}
for cat in categories_list:
    query_url = base_url + f'/v1/browse/categories/{cat}/playlists?limit=5'
    playlists = []
    results = requests.get(query_url, headers=headers_dict).json()
    try: 
        for id, playlist in enumerate(results['playlists']['items']):
            playlists.append(playlist['id'])
        if len(playlists) > 0:
            category_playlist_ids[cat] = playlists
    except:
        print(f'No playlists for category: {cat}')
print(f'Playlists found for {len(category_playlist_ids)} categories.')
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

No playlists for category: regional_mexican
No playlists for category: music_of_mexico
No playlists for category: radar
No playlists for category: student
No playlists for category: j_tracks
No playlists for category: flamenco
No playlists for category: popculture
No playlists for category: thirdparty
No playlists for category: anime
No playlists for category: sessions
No playlists for category: blackhistorymonth
No playlists for category: colombian
No playlists for category: brazilian
Playlists found for 37 categories.
Run time: 8.588656902313232 seconds


In [5]:
# Create a DataFrame of song names, artists, categories, and genres
t0 = time.time()
song_genres_df = pd.DataFrame(columns=['song', 'song_id', 'artist', 'artist_id', 'category'])
song_genres_df
index = 0
for category in category_playlist_ids:
    songs_list = []
    # Determine how many songs to get from each playlist
    playlists = category_playlist_ids[category]
    len_limit = {1: 25, 2:10, 3: 8, 4: 6, 5: 5}
    length = len(category_playlist_ids[category])
    if length == 0:
        print(category)
    limit = len_limit[length]
    for playlist_id in playlists:
        # Get songs from each playlist
        query_url = base_url + f'/v1/playlists/{playlist_id}/tracks?limit={limit}'
        results = requests.get(query_url, headers=headers_dict).json()
        try:
            # Get the song, artist, and genres for each song
            for id, item in enumerate(results['items']):
                song = item['track']['name']
                song_id = item['track']['id']
                artist = item['track']['artists'][0]['name']
                artist_id = item['track']['artists'][0]['id']
                song_genres_df.loc[index] = [song, song_id, artist, artist_id, category]
                index+=1
        except:
            print(f'No results for playlist {playlist_id}')

t1 = time.time()
print(f'Run time: {t1-t0} seconds')
song_genres_df.head()


No results for playlist 37i9dQZF1EOwyah3Akjjez
No results for playlist 37i9dQZF1DX7YCknf2jT6s
Run time: 34.661799907684326 seconds


,song,song_id,artist,artist_id,category
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists


In [6]:
# Create a dictionary to check how many songs there are for each category
t0 = time.time()
songs_per_cat = {}
for category in category_playlist_ids:
    songs_per_cat[category] = len(song_genres_df[song_genres_df['category']==category])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
songs_per_cat

Run time: 0.039868831634521484 seconds


{'toplists': 25,
 '2020': 22,
 'holidays': 25,
 'at_home': 25,
 'wellness': 25,
 'pop': 20,
 'latin': 25,
 'instrumental': 25,
 'family': 25,
 'sleep': 24,
 'workout': 25,
 'chill': 25,
 'indie_alt': 25,
 'metal': 25,
 'rock': 25,
 'hiphop': 25,
 'edm_dance': 25,
 'party': 25,
 'decades': 25,
 'romance': 25,
 'mood': 25,
 'focus': 25,
 'dinner': 25,
 'gaming': 25,
 'inspirational': 25,
 'pride': 25,
 'kpop': 25,
 'soul': 25,
 'roots': 25,
 'caribbean': 25,
 'jazz': 24,
 'classical': 25,
 'punk': 20,
 'rnb': 25,
 'blues': 25,
 'funk': 25,
 'travel': 25}

In [7]:
# Create a dictionary of genres for each artist
t0 = time.time()
artist_genres = {}
count = 0
for artist_id in song_genres_df['artist_id'].unique():
    query_url = base_url + f'/v1/artists/{artist_id}'
    artist = requests.get(query_url, headers=headers_dict).json()
    genres = artist['genres']
    genres = list(set([genre.split()[0] for genre in genres]))
    artist_genres[artist_id] = genres
    if genres == []:
        artist_name = artist['name']
        count+=1

t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'No genres found for {count} artists.')
len(artist_genres)

Run time: 106.29341506958008 seconds
No genres found for 108 artists.


746

In [8]:
# Add a column of artist genres
t0 = time.time()
genre_column = []
for artist_id in song_genres_df['artist_id']:
    genre_column.append(artist_genres[artist_id])
song_genres_df['genres'] = genre_column

t1 = time.time()
print(f'Run time: {t1-t0} seconds')
song_genres_df.head()

Run time: 0.0027539730072021484 seconds


,song,song_id,artist,artist_id,category,genres
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"[pop, viral, canadian, post-teen, dance]"
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"[pop, electropop]"
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"[pop, uk, dance]"
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"[pop, post-teen]"
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"[lgbtq+, pop, queer, country]"


In [13]:
# Create a function that adds a column of filtered genres
def filter_genres(song_genres_df):
    # Create a non-destructive copy
    song_genres_df = pd.DataFrame(song_genres_df)
    
    # Create a list of all genres
    genre_list = []
    genre_columns = song_genres_df['genres']
    for artist_genres in genre_columns:
        genre_list.extend(artist_genres)
    genre_list = list(set(genre_list))
    print(f'{len(genre_list)} unique genres found.')
    
    # Find most popular genres
    popular_genres = dict.fromkeys(genre_list, 0)
    for artist_genres in genre_columns:
        for genre in artist_genres:
            popular_genres[genre]+=1
    popular_genres = dict(sorted(popular_genres.items(), key=lambda kv: kv[1], reverse=True))
    
    # Create a list of genres that applies to all songs
    t0 = time.time()
    common_genres = []
    song_index_list = []
    for index, row in song_genres_df.iterrows():
        if row['genres'] == []:
            song_index_list.append(index)
    counter = 0
    while len(set(song_index_list)) < len(song_genres_df):
        new_genre = list(popular_genres.keys())[counter]
        new_indeces = []
        for index, row in song_genres_df.iterrows():
            if (new_genre in row['genres']) & (index not in song_index_list):
                new_indeces.append(index)
        if new_indeces != []:
            song_index_list.extend(new_indeces)
            common_genres.append(new_genre)
        counter+=1
    t1 = time.time()
    print(f'Run time to find common genres: {t1-t0} seconds')
    print(f'All songs are encompassed by {len(common_genres)} genres.')
    
    # Add a column of filtered genres
    filtered_genres = []
    for artist_genres in genre_columns:
        filtered = []
        for genre in artist_genres:
            if genre in common_genres:
                filtered.append(genre)
        filtered_genres.append(filtered)
    song_genres_df['filtered_genres'] = filtered_genres
    
    return song_genres_df


In [14]:
# Run the filter_genres function
song_genres_df = filter_genres(song_genres_df)
song_genres_df.head()

385 unique genres found.
Run time to find common genres: 35.76010608673096 seconds
All songs are encompassed by 111 genres.


,song,song_id,artist,artist_id,category,genres,filtered_genres
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"[pop, viral, canadian, post-teen, dance]","[pop, viral, canadian, dance]"
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"[pop, electropop]",[pop]
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"[pop, uk, dance]","[pop, uk, dance]"
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"[pop, post-teen]",[pop]
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"[lgbtq+, pop, queer, country]","[pop, country]"


In [15]:
# Create lists of audio features
t0 = time.time()
audio_features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                  'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
                  'duration_ms', 'time_signature']
danceability = [] 
energy = [] 
key = []
loudness = [] 
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []
time_signature = []
for song_id in song_genres_df['song_id']:
    query_url = base_url + f'/v1/audio-features/{song_id}'
    features = requests.get(query_url, headers=headers_dict).json()
    danceability.append(features['danceability'])
    energy.append(features['energy'])
    key.append(features['key'])
    loudness.append(features['loudness'])
    mode.append(features['mode'])
    speechiness.append(features['speechiness'])
    acousticness.append(features['acousticness'])
    instrumentalness.append(features['instrumentalness'])
    liveness.append(features['liveness'])
    valence.append(features['valence'])
    tempo.append(features['tempo'])
    duration_ms.append(features['duration_ms'])
    time_signature.append(features['time_signature'])

t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(f'Audio features added for {len(danceability)} songs.')

Run time: 129.6081621646881 seconds
Audio features added for 910 songs.


In [16]:
# Add columns of audio features
t0 = time.time()
song_genres_df['audio_ft_danceability'] = danceability
song_genres_df['audio_ft_energy'] = energy
song_genres_df['audio_ft_key'] = key
song_genres_df['audio_ft_loudness'] = loudness
song_genres_df['audio_ft_mode'] = mode
song_genres_df['audio_ft_speechiness'] = speechiness
song_genres_df['audio_ft_acousticness'] = acousticness
song_genres_df['audio_ft_instrumentalness'] = instrumentalness
song_genres_df['audio_ft_liveness'] = liveness
song_genres_df['audio_ft_valence'] = valence
song_genres_df['audio_ft_tempo'] = tempo
song_genres_df['audio_ft_duration_ms'] = duration_ms
song_genres_df['audio_ft_time_signature'] = time_signature

t1 = time.time()
print(f'Run time: {t1-t0} seconds')
song_genres_df.head()


Run time: 0.023520946502685547 seconds


,song,song_id,artist,artist_id,category,genres,filtered_genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature
0,Monster (Shawn Mendes & Justin Bieber),2Z8yfpFX0ZMavHkcIeHiO1,Shawn Mendes,7n2wHs1TKAczGzO7Dd2rGr,toplists,"[pop, viral, canadian, post-teen, dance]","[pop, viral, canadian, dance]",0.652,0.383,2,-7.076,0,0.0516,0.06760,0.00,0.0828,0.549,145.765,178994,4
1,Therefore I Am,54bFM56PmE4YLRnqpW6Tha,Billie Eilish,6qqNVTkY8uBg9cP3Jd7DAH,toplists,"[pop, electropop]",[pop],0.889,0.340,11,-7.773,0,0.0697,0.21800,0.13,0.0550,0.716,94.009,174321,4
2,Levitating (feat. DaBaby),463CkQjx2Zk1yXoBuierM9,Dua Lipa,6M2wZ9GZgrQXHCFfjv46we,toplists,"[pop, uk, dance]","[pop, uk, dance]",0.702,0.825,6,-3.787,0,0.0601,0.00883,0.00,0.0674,0.915,102.977,203064,4
3,positions,7igeByaBM0MgGsgXtNxDJ7,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,toplists,"[pop, post-teen]",[pop],0.736,0.802,0,-4.759,1,0.0864,0.46800,0.00,0.0940,0.675,144.005,172325,4
4,HOLIDAY,6zFMeegAMYQo0mt8rXtrli,Lil Nas X,7jVv8c5Fj3E9VhNjxT4snq,toplists,"[lgbtq+, pop, queer, country]","[pop, country]",0.810,0.511,5,-6.924,0,0.1640,0.12000,0.00,0.0832,0.837,151.947,154998,4


In [17]:
# Save DataFrame to CSV
song_genres_df.to_csv('../Data/song_genres.csv', index=False)

In [ ]:
# Create a dictionary of audio features
#t0 = time.time()
#audio_features = defaultdict(list)
#features_list = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
#                  'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 
#                  'duration_ms', 'time_signature']
#for song_id in song_genres_df['song_id']:
#    query_url = base_url + f'/v1/audio-features/{song_id}'
#    features = requests.get(query_url, headers=headers_dict).json()
#    for feat in features_list:
#        audio_features[feat].append(features[feat])
#audio_features = dict(audio_features)

#t1 = time.time()
#print(f'Run time: {t1-t0} seconds')
#print(f'Audio features added for {len(danceability)} songs.')
#audio_features

In [ ]:
# Create a list of all genres
genre_list = []
for artist_genres in genre_column:
    genre_list.extend(artist_genres)
genre_list = list(set(genre_list))
len(genre_list)

In [ ]:
genre_list = ['afro' if genre.startswith('afro') else genre for genre in genre_list]
genre_list = ['arab' if genre.startswith('arab') else genre for genre in genre_list]
genre_list = ['bass' if genre.startswith('bass') else genre for genre in genre_list]
genre_list = ['british' if genre.startswith('brit') else genre for genre in genre_list]
genre_list = ['bubble' if genre.startswith('bubble') else genre for genre in genre_list]
genre_list = ['chill' if genre.startswith('chill') else genre for genre in genre_list]
genre_list = ['dance' if genre.startswith('dance') else genre for genre in genre_list]
genre_list = ['electric' if genre.startswith('electr') else genre for genre in genre_list]
genre_list = ['euro' if genre.startswith('euro') else genre for genre in genre_list]
genre_list = ['folk' if genre.startswith('folk') else genre for genre in genre_list]
genre_list = ['indie' if genre.startswith('indie') else genre for genre in genre_list]
genre_list = ['neo-classical' if genre.startswith('neoclassical') else genre for genre in genre_list]
genre_list = ['post' if genre.startswith('post') else genre for genre in genre_list]
genre_list = ['reggae' if genre.startswith('reggae') else genre for genre in genre_list]
genre_list = ['rock' if genre.startswith('rock') else genre for genre in genre_list]
genre_list = ['south' if genre.startswith('south') else genre for genre in genre_list]
genre_list = list(set(genre_list))
print(len(genre_list))

In [ ]:
#  Add the clean movie function that takes in the argument, "movie"
def clean_movie(movie):
    # Create a non-destructive copy
    movie = dict(movie)
    alt_titles = {}
    # Combine alternate titles into one list
    for key in ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French',
               'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 'Literally',
               'Mandarin', 'McCune–Reischauer', 'Original title', 'Polish',
               'Revised Romanization', 'Romanized', 'Russian',
               'Simplified', 'Traditional', 'Yiddish']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        
    # Merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')    
    return movie

In [ ]:
# Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load(wiki_data, kaggle_data, rating_data):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames
    kaggle_metadata = pd.read_csv(kaggle_data, low_memory=False)
    ratings = pd.read_csv(rating_data)

    # Open and read the Wikipedia data JSON file
    with open(wiki_data, mode='r') as file:
        wiki_movies_raw = json.load(file)

    # Write a list comprehension to filter out TV shows
    wiki_movies = [movie for movie in wiki_movies_raw
            if ('Director' in movie or 'Directed by' in movie)
                and ('imdb_link' in movie) 
                and ('No. of episodes' not in movie)]

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie
    clean_movies = [clean_movie(movie) for movie in wiki_movies]    
    
    # Read in the cleaned movies list from Step 4 as a DataFrame
    wiki_movies_df = pd.DataFrame(clean_movies)

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame
    wiki_columns_to_keep = [col for col in wiki_movies_df.columns if wiki_movies_df[col].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # Create a variable that will hold the non-null values from the “Box office” column
    box_office = wiki_movies_df['Box office'].dropna()
    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Write a regular expression to match the six elements of "form_one" of the box office data
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    # Write a regular expression to match the three elements of "form_two" of the box office data
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?![\s[mb]illion])'

    # Add the parse_dollars function
    def parse_dollars(s):
        # if s is not a string, return NaN
        if type(s) != str:
            return np.nan
    
        # If input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            # convert to float and multiply by a million
            value = float(s) * 10**6
            # return value
            return value
    
        # If input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]', '', s)
            # convert to float and multiply by a billion
            value = float(s) * 10**9
            # return value
            return value
    
        # If input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            # remove dollar sign and commas
            s = re.sub('\$|,', '', s)
            # convert to float
            value = float(s)
            # return value
            return value
    
        # Otherwise, return NaN
        else:
            return np.nan
    
    # Clean the box office column in the wiki_movies_df DataFrame
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # Clean the budget column in the wiki_movies_df DataFrame
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-–](?![a-z])', '$', regex=True)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE)
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # Clean the release date column in the wiki_movies_df DataFrame
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|Feburary|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|Feburary|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)[0], infer_datetime_format=True)

    # Clean the running time column in the wiki_movies_df DataFrame
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s?ho?u?r?s?\s?(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
         
    # Clean the Kaggle metadata
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult', axis='columns')
    #kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    #kaggle_metadata['video'] = kaggle_metadata['video'] == True
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    # Merged the two DataFrames into the movies DataFrame
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki', '_kaggle'])

    # Drop unnecessary columns from the merged DataFrame
    #movies_df = movies_df.drop(movies_df[(movies_df['release_date_wiki'] > '1995-01-01') & (movies_df['release_date_kaggle'] < '1970-01-01')].index)
    movies_df.drop(columns=['title_wiki', 'release_date_wiki', 'Language', 'Production company(s)'], inplace=True)

    # Add in the function to fill in the missing Kaggle data
    def fill_missing_kaggle_data(df, kaggle_col, wiki_col):
        df[kaggle_col] = df.apply(
            lambda row: row[wiki_col] if row[kaggle_col] == 0 else row[kaggle_col], axis=1)
        df.drop(columns=wiki_col, inplace=True)

    # Call the function in Step 5 with the DataFrame and columns as the arguments
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')

    # Filter the movies DataFrame for specific columns
    movies_df = movies_df.loc[:, [
        'imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
        'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
        'genres','original_language','overview','spoken_languages','Country', 'production_companies', 
        'production_countries','Distributor', 'Producer(s)','Director','Starring','Cinematography',
        'Editor(s)','Writer(s)','Composer(s)','Based on']]

    # Rename the columns in the movies DataFrame
    movies_df.rename({
        'id':'kaggle_id',
        'title_kaggle':'title',
        'url':'wikipedia_url',
        'budget_kaggle':'budget',
        'release_date_kaggle':'release_date',
        'Country':'country',
        'Distributor':'distributor',
        'Producer(s)':'producers',
        'Director':'director',
        'Starring':'starring',
        'Cinematography':'cinematography',
        'Editor(s)':'editors',
        'Writer(s)':'writers',
        'Composer(s)':'composers',
        'Based on':'based_on'},
        axis='columns', inplace=True)
    
    # Transform and merge the ratings DataFrame
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
    rating_counts = ratings.groupby(['movieId', 'rating'], as_index=False).count() \
        .rename(columns={'userId': 'count'}) \
        .pivot(index='movieId', columns='rating', values='count')
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')
    movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)

    return wiki_movies_df, movies_with_ratings_df, movies_df


In [ ]:
# Create the path to your file directory and variables for the three files
file_dir = '../Data'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data
ratings_file = f'{file_dir}/ratings.csv'

In [ ]:
# Set the three variables equal to the function created in function_test.ipynb ?
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file, kaggle_file, ratings_file)


In [ ]:
# Set the DataFrames from the return statement equal to the file variables
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [ ]:
# Check the wiki_movies_df DataFrame
wiki_movies_df.head()

In [ ]:
# Check the movies_with_ratings_df DataFrame
movies_with_ratings_df.head()

In [ ]:
# Check the movies_df DataFrame
movies_df.head()